[Ditherer](https://github.com/adlrwbr/Tessellate)

[Solver](https://pypi.org/project/kociemba/)

#### Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install --upgrade Pillow kociemba plotly

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/content/drive/MyDrive/Rubik's Mosaic


In [4]:
import kociemba as kc
from PIL import Image
import matplotlib.pyplot as plt
from enum import Enum
import numpy as np
import plotly
import plotly.graph_objs as go
import random
import time
import itertools
import math
from tqdm import tqdm

#### Pancake Dither

In [1]:
"""
The MIT License (MIT)

Copyright (c) 2015 Mat Leonard

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
THE SOFTWARE.
"""
class Vector(object):
    def __init__(self, *args):
        """ Create a vector, example: v = Vector(1,2) """
        if len(args) == 0:
            self.values = (0, 0)
        else:
            self.values = args

    def norm(self):
        """ Returns the norm (length, magnitude) of the vector """
        return math.sqrt(sum(comp**2 for comp in self))

    def floor(self):
        """ Round all components down to the nearest integer """
        return Vector(*tuple(int(comp) for comp in self))
    
    def tuple(self):
        """ Return a tuple representation of the Vector for hashing """
        return tuple(comp for comp in self)

    def __mul__(self, other):
        """ Returns the dot product of self and other if multiplied
            by another Vector.  If multiplied by an int or float,
            multiplies each component by other.
        """
        if type(other) == type(self):
            return self.inner(other)
        elif type(other) == type(1) or type(other) == type(1.0):
            product = tuple(a * other for a in self)
            return Vector(*product)

    def __add__(self, other):
        """ Returns the vector addition of self and other """
        added = tuple(a + b for a, b in zip(self, other))
        return Vector(*added)

    def __sub__(self, other):
        """ Returns the vector difference of self and other """
        subbed = tuple(a - b for a, b in zip(self, other))
        return Vector(*subbed)

    def __iter__(self):
        return self.values.__iter__()

    def __repr__(self):
        return str(self.values)

    def __eq__(self, other) -> bool:
        return all(s == o for s,o in zip(self, other))

def gradient(img, out, thresholds=[91, 121, 151, 180, 255], colors=[
    # Blue
    Vector(0, 0, 255),
    # Red
    Vector(255, 0, 0),
    # Orange
    Vector(255, 165, 0),
    # Yellow
    Vector(255, 255, 0),
    # White
    Vector(255, 255, 255),
], background_color = Vector(255, 255, 255)):
  def pixelToColor(pixel):
      if pixel_map[x,y][-1] == 0: return background_color.tuple()
      brightness = sum(pixel[:2])/3
      for c, t in zip(colors, sorted(thresholds)):
          if brightness <= t:
            # print(c.tuple())
              return c.tuple()

  with Image.open(img) as img:
      # grayscale = sorted(sum(pixel[:3])/3 for pixel in list(img.getdata()))
      width, height = img.size
      pixel_map = img.load()
      for y in range(height):
          for x in range(width):
              pixel_map[x,y] = pixelToColor(pixel_map[x,y])
      img.save(out)

#Input custom colors in URFDLB order to match lighting of final mosaic location
def FloydSteinbergPancakeDither(
    img1, img2, bmp1, bmp2, size, 
    colors=[
    # White
    Vector(255, 255, 255),
    # Red
    Vector(255, 0, 0),
    # Green
    Vector(0, 255, 0),
    # Yellow
    Vector(255, 255, 0),
    # Orange
    Vector(255, 165, 0),
    # Blue
    Vector(0, 0, 255)]):
    def opposite(c): 
        return colors[(colors.index(c) + 3)%len(colors)]
    
    def adjacent(c): 
        return [a for a in colors if a not in [c, opposite(c)]]

    def getPalette(colors, counts, piece_type):
        if piece_type == 1: return colors
        palette = []
        CoP = counts[piece_type]
        for c in colors:
            adjacent_faces = adjacent(c)
            color_count = CoP[c.tuple()]
            # This part should probably look nicer
            if color_count < 4 \
              and all(color_count + CoP[adj.tuple()] < (9 - piece_type) for adj in adjacent_faces) \
              and all(color_count + CoP[a1.tuple()] + CoP[a2.tuple()] < 13 - (2 * piece_type) \
                      for a1, a2 in zip(adjacent_faces, adjacent_faces[1:]+[adjacent_faces[0]])) \
              and sum(piece[c.tuple()] for piece in counts.values()) < 9:
                  palette.append(c)
        return palette

    cubes = []

    with Image.open(img1).resize(size, Image.Resampling.LANCZOS) as img1:
        with Image.open(img2).resize(size, Image.Resampling.LANCZOS).transpose(
            method=Image.Transpose.FLIP_LEFT_RIGHT) as img2: # Is this spaced properly?
            # get dimensions (Probably slash)
            width, height = img1.size

            # load all pixels from image (Probably slash)
            pixel_map1, pixel_map2 = img1.load(), img2.load()

            sticker_counts = {}
            # dithering algorithm (See if this is okay)
            for y in tqdm(range(height)):
                for x in range(width):
                    cube_idx = (x//3,y//3)
                    if y%3 == 0 and x%3 == 0:
                        sticker_counts[cube_idx] = {
                            1:{c.tuple(): 0 for c in colors},
                            2:{c.tuple(): 0 for c in colors}, 
                            3:{c.tuple(): 0 for c in colors}
                            }
                    piece_type = 1 if x%3 == 1 and y%3 == 1 else 2 if x%3 == 1 or y%3 == 1 else 3
                    counts = sticker_counts[cube_idx]

                    palette = getPalette(colors, counts, piece_type)

                    im1_color = dither(pixel_map1, x, y, palette, width, height)
                    counts[piece_type][im1_color.tuple()] += 1

                    centers = [Vector(*key) for key, val in counts[1].items() if val]

                    if piece_type == 1: palette = [opposite(im1_color)]
                    elif x%3 == 2 and y%3 == 2 and sum(counts[3][c.tuple()] for c in centers) == 7:
                        palette = getPalette(centers, counts, piece_type)
                    else:
                        palette = getPalette(colors, counts, piece_type)

                    if x%3 == 2 and y%3 == 2:
                        palette, cube = ensureSolvability(palette, colors, x, y, pixel_map1, pixel_map2)
                        cubes.append(cube)

                    im2_color = dither(pixel_map2, x, y, palette, width, height)
                    counts[piece_type][im2_color.tuple()] += 1

                    temp = pixel_map1
                    pixel_map1 = pixel_map2
                    pixel_map2 = temp

            img1.save(bmp1)
            img2.save(bmp2)
    return cubes

def ensureSolvability(palette, colors, x, y, pixel_map1, pixel_map2):
    colorsToLetters = {c.tuple(): letter for c, letter in zip(colors, 'URFDLB')}
    minX, minY = x - (x%3), y - (y%3)
    xVals = list(range(minX,minX+3)) * 3
    yVals = [minY] * 3 + [minY+1] * 3 + [minY+2] * 3
    top = [colorsToLetters.get(pixel_map1[i,j], Cube.wildcard) for i,j in zip(xVals, yVals)]
    bot = [colorsToLetters.get(pixel_map2[i,j], Cube.wildcard) for i,j in zip(xVals, yVals[::-1])]
    cubestring = top + [Cube.wildcard]*18 + bot + [Cube.wildcard]*18
    cubestring[29] = '?'
    c = Cube(''.join(cubestring))
    #decolor to make cube correspond to input array order
    cube = c.getDecoloredFinalCubestring() if c.getResult()[0] else None
    palette = [Vector(*next(key for key, val in colorsToLetters.items() if val == cube[29]))]
    return palette, c

def dither(pixel_map, x, y, palette, width, height):
    oldpixel = Vector(*pixel_map[x, y])
    newpixel = sorted(palette, key=lambda c: (c - oldpixel).norm())[0]
    pixel_map[x, y] = newpixel.tuple()
    quant_error = oldpixel - newpixel
    
    def addError(x, y, fraction):
        pixel_map[x, y] = (Vector(*pixel_map[x, y]) + quant_error * fraction).floor().tuple()

    if (x + 1 < width): addError(x + 1, y, 7 / 16)
    if (y + 1 < height):
        if (x > 0):
            addError(x - 1, y + 1, 3 / 16)
        addError(x, y + 1, 5 / 16)
        if (x + 1 < width):
            addError(x + 1, y + 1, 1 / 16)

    return newpixel

In [ ]:
def showImage(fname):
    with Image.open(fname) as im:
        fig, ax = plt.subplots(figsize=(18, 10))
        ax.imshow(im)
        plt.tight_layout()

#### Inference

In [ ]:
# Remove this later--just saying I need to fix indenting and stuff here
class Status(Enum):
  IMP_STICKERS = 'More than 9 stickers of a single color. Get more stickers.'
  PERM_STICKERS = 'Abnormally colored pieces. Peel off and rearrange stickers.'
  PERM_PIECES = 'Unsolvable'
  SUPPOSE = 'No suppositions work'
  SOLVABLE = 'Congrats, solvable cube'
  CHIRALITY = 'Wrong chirality'
  DUPLICATE_COLORS = 'Duplicate colors on one piece'
  BAD_CENTERS = 'Centers impossible'

# labeling of corners and indices 
pieceToStringIdx = {
  'U': (4,),
  'R': (13,),
  'F': (22,),
  'D': (31,),
  'L': (40,),
  'B': (49,),
  'UB': (1, 46),
  'UL': (3, 37),
  'UR': (5, 10),
  'UF': (7, 19),
  'DF': (28, 25),
  'DL': (30, 43),
  'DR': (32, 16),
  'DB': (34, 52),
  'FL': (21, 41),
  'FR': (23, 12),
  'BR': (48, 14),
  'BL': (50, 39),
  'UFL': (6, 18, 38),
  'URF': (8, 9, 20),
  'DFR': (29, 26, 15),
  'DLF': (27, 44, 24),
  'ULB': (0, 36, 47),
  'UBR': (2, 45, 11),
  'DRB': (35, 17, 51),
  'DBL': (33, 53, 42)
}

oppositeFace = {}
for s in ['FB','UD','LR']:
  oppositeFace[s[0]] = s[1]
  oppositeFace[s[1]] = s[0]

def filterPieces(length):
  return [p for p in pieceToStringIdx.keys() if len(p)==length]

def checkChirality(corner):
  return any(corner in cycle for cycle in cyclics)

def getTriplet(edge):
  cycle = [c for c in cyclics if edge in c][0]
  return cycle[(cycle.find(edge)+2)%3]

Pieces = {'Center': filterPieces(1),
          'Edge': filterPieces(2),
          'Corner': filterPieces(3)}

getPiecesByLen = (0, Pieces['Center'], Pieces['Edge'], Pieces['Corner'])

cyclics = [s + s[:2] for s in Pieces['Corner']]

class Cube:
  wildcard = '?'
  stickers_per_face = 9
  faces_per_cube = 6

  def __init__(self, cubestring: str, wait_time=1):
    assert len(cubestring) == Cube.stickers_per_face * Cube.faces_per_cube, \
      'Cube string for 3x3x3 must be 54 characters.'
    assert not any(cubestring.count(c) > Cube.stickers_per_face for c in Pieces['Center']), \
      Status.IMP_STICKERS.value

    self.wait_time = wait_time

    self.result = [True,True]
    
    locationsToPieces = {key: filterPieces(len(key)) for key in pieceToStringIdx.keys()}
    self.cubestring = cubestring
    pruning = True
    while pruning:
      pruning = False
      for key, val in pieceToStringIdx.items():
        colors = [cubestring[v] for v in val if cubestring[v] != Cube.wildcard]
        if len(colors) == 3: assert checkChirality(''.join(colors)), Status.CHIRALITY.value
        
        assert all(colors.count(x) == 1 for x in colors), Status.DUPLICATE_COLORS.value

        prevNumOptions = len(locationsToPieces[key])
        narrowed = [option for option in locationsToPieces[key] if all(c in option for c in colors)]

        if len(narrowed) < prevNumOptions: pruning = True
        if len(narrowed) == 1: self.inferenceCascade(key, narrowed, locationsToPieces)

        assert len(narrowed), Status.PERM_STICKERS.value + ': ' + str(len(key))
        
        locationsToPieces[key] = narrowed

    # save post-pruning locationsToPieces (questionable deep copy)
    self.start_time = time.perf_counter()

    kc_solvable = False
    # while time.perf_counter() - self.start_time < self.wait_time and not kc_solvable:

    suppose = {key: val[:] for key, val in locationsToPieces.items()}
    unassigned = next((key for key, val in suppose.items() if len(val) > 1), False)
    if unassigned:
      valid_supposition, suppose = self.suppose(unassigned, suppose,0)
      # if not valid_supposition: print(cubestring)
      assert valid_supposition, Status.SUPPOSE.value
    # print('supposing worked')
    
    # to canonical orientation and no more lists
    self.translate = ''.join(suppose[c][0] for c in 'URFDLB')
    self.recolorPieces(suppose)
    recolored_cubestring = self.recolorString(self.cubestring, self.translate)
    # print(cubestring, recolored_cubestring)
    
    final_cubestring = [''] * (Cube.stickers_per_face * Cube.faces_per_cube)

    for key, val in suppose.items():
      piece = val
      piece_type = len(key)
      if piece_type == 1:
        final_cubestring[pieceToStringIdx[key][0]] = piece
      else:
        idxToColor = [(i, recolored_cubestring[i]) for i in pieceToStringIdx[key]]

        idxToColor = sorted([(idxToColor * 2)[offset: offset+piece_type] for offset in range(piece_type)],
                            key = lambda perm: ''.join([pair[1] for pair in perm][::-1])
                            )[0]
        # print(idxToColor)
        first_color = idxToColor[0][1]
        colors = piece if first_color == Cube.wildcard else (piece * 2)[piece.find(first_color):]
        for pair, c in zip(idxToColor, colors): final_cubestring[pair[0]] = c

    perm_parity = self.getPermParity(suppose)

    # fixes perm parity
    if not perm_parity:
      for e1, e2 in itertools.combinations(getPiecesByLen[2], 2):
        idx1, idx2 = pieceToStringIdx[e1], pieceToStringIdx[e2]
        if all(self.cubestring[i] == Cube.wildcard for i in idx1 + idx2):
          swapped = {key: val[:] for key, val in suppose.items()}
          swapped[e1], swapped[e2] = swapped[e2], swapped[e1]
          if self.getPermParity(swapped):
            for i,j in zip(idx1,idx2):
              temp = final_cubestring[i]
              final_cubestring[i] = final_cubestring[j]
              final_cubestring[j] = temp
            perm_parity = True
            break

    edge_parity = self.getTwistParity(suppose, final_cubestring, 2)

    # fixes edge parity
    if not edge_parity:
      for edge in Pieces['Edge']:
        idx = pieceToStringIdx[edge]
        if all(self.cubestring[i]=='?' for i in idx):
          final_cubestring[idx[0]], final_cubestring[idx[1]] = \
          final_cubestring[idx[1]], final_cubestring[idx[0]]
          edge_parity = True
          break

    # print('Parities', perm_parity, edge_parity, corner_parity)
    corner_parity = self.getTwistParity(suppose, final_cubestring, 3)
    corner_parity_value = self.getTwistParity(suppose, final_cubestring, 3, returnIsCorrect=False)

    if not corner_parity:
      for corner in Pieces['Corner']:
        idx = pieceToStringIdx[corner]
        if all(self.cubestring[i]=='?' for i in idx):
          colors = ''.join(final_cubestring[i] for i in pieceToStringIdx[corner])
          for offset, i in enumerate(pieceToStringIdx[corner]): 
            final_cubestring[i] = colors[(offset + corner_parity_value) % 3]
          corner_parity = True
          break
          

    solvable = perm_parity and edge_parity and corner_parity

    self.parities = [perm_parity, edge_parity, corner_parity]

    # print(''.join(final_cubestring), self.parities)

    self.final_cubestring = ''.join(final_cubestring)

    kc_solvable = True
    try:
      # print(final_cubestring)
      self.solution = self.reverseInstructions(kc.solve(self.final_cubestring))
    except Exception as e:
      # print(e)
      kc_solvable = False

    # print(kc_solvable)
    
    self.result = (kc_solvable, kc_solvable == solvable)
    # print(self.getDecoloredFinalCubestring(), cubestring)
  
  def getFinalCubestring(self): return self.final_cubestring
  def getSolution(self): return self.solution
  def getDecoloredFinalCubestring(self):
    return self.recolorString(self.final_cubestring, 'URFDLB', self.translate)

  def reverseInstructions(self, ins):
    return ' '.join([s if '2' in s else s+"'" if "'" not in s else s[:-1] \
    for s in ins.split()[::-1]])

  def suppose(self, collapseKey, suppose, i):
    if time.perf_counter() - self.start_time > self.wait_time: return False, suppose
    for j, option in enumerate(suppose[collapseKey]):
      # print(i, j, {key:val for key,val in suppose.items() if len(val) != 1})
      # if i > 11: print('possibilities',collapseKey, suppose[collapseKey])
      # print('testing option', option, 'for piece', collapseKey)
      # if j == 0:
      #   status.append(0)
      # status[i] = j
      # out.update(IPython.display.Pretty(''.join(str(s) for s in status)))

      collapseVal = [option]
      branch = {key: val[:] for key, val in suppose.items()}
      branch[collapseKey] = collapseVal
      try:
        branch = self.inferenceCascade(collapseKey, collapseVal, branch)
        unassigned = sorted([key for key, val in branch.items() if len(val) > 1], key = lambda x: len(branch[x]))
        if not unassigned: return True, branch
        else:
          valid_s, s = self.suppose(unassigned[0], branch, i+1)
          if valid_s: return True, s
      except Exception as e:
        # print(i, 'branch failed')
        continue
    # print('no sup')
    return False, suppose

  def getResult(self):
    return self.result
  
  def getAverageSuperposition(self):
    return sum(len(val) for val in self.suppose.values())/len(self.locationsToPieces)

  def getTwistParity(self, suppose, cubestring, piece_type, returnIsCorrect=True):
    parity = 0
    for piece in getPiecesByLen[piece_type]:
      colors = ''.join(cubestring[i] for i in pieceToStringIdx[piece])
      poles = 'FB' if piece_type == 2 and not any(p in colors for p in 'UD') else 'UD'
      for p in poles:
        if p in colors: parity += colors.find(p)
    parity %= piece_type
    return parity == 0 if returnIsCorrect else parity

  def getPermParity(self, suppose):
    return self.getCycles(getPiecesByLen[3], suppose) == \
    self.getCycles(getPiecesByLen[2], suppose)

  def getCycles(self, piece_type, suppose):
    seen = set()
    evenCycles = 0
    for piece in piece_type:
      if piece not in seen:
        cycle = 1 # from quora
        location = suppose[piece]
        while location != piece:
          cycle += 1
          seen.add(location)
          location = suppose[location]
        evenCycles += 1 - cycle % 2
    return evenCycles % 2

  def removePossibilities(self, key, final_piece, locationsToPieces):
    narrowed = []
    for piece in getPiecesByLen[len(key)]:
      if piece != key and final_piece in locationsToPieces[piece]:
        locationsToPieces[piece].remove(final_piece)
        remaining = locationsToPieces[piece]
        if len(remaining) == 1: narrowed.append((piece,remaining))
        # if not locationsToPieces[piece]: print('no more possy')
        assert locationsToPieces[piece], 'Inference dead end'
    for piece, remaining in narrowed:
      #print('RP trace', key, final_piece) #
      self.inferenceCascade(piece, remaining, locationsToPieces)
  
  def inferenceCascade(self, key, narrowed, locationsToPieces):
    final_piece = narrowed[0]
    self.removePossibilities(key, final_piece, locationsToPieces)

    if len(final_piece) == 1:
      opKey, opColor = oppositeFace[key], oppositeFace[final_piece]
      locationsToPieces[opKey] = [opColor]
      self.removePossibilities(opKey, opColor, locationsToPieces)
      
      # if two adjacent centers are set, the rest are determined by chirality and opposites
      for loc0, loc1 in [e for e in getPiecesByLen[2] if final_piece in e]:
        c0, c1 = locationsToPieces[loc0], locationsToPieces[loc1]
        if len(c0) + len(c1) == 2:
          loc2, c2 = getTriplet(loc0+loc1), getTriplet(c0[0]+c1[0])
          loc3, c3 = oppositeFace[loc2], oppositeFace[c2]
          assert c2 in locationsToPieces[loc2] and c3 in locationsToPieces[loc3], \
            Status.BAD_CENTERS.value
          locationsToPieces[loc2] = [c2]
          locationsToPieces[loc3] = [c3]
          self.removePossibilities(loc2, c2, locationsToPieces)
          self.removePossibilities(loc3, c3, locationsToPieces)

    return locationsToPieces

  def recolorString(self, string, fromColoring, toColoring='URFDLB'):
    for i, t in enumerate(fromColoring):
      string = string.replace(t, str(i))
    for i, c in enumerate(toColoring):
      string = string.replace(str(i), c)
    return string

  def recolorPieces(self, suppose):
    translate = {suppose[c][0]: c for c in "URFDLB"}
    for key, val in suppose.items():
      suppose[key] = next(key for key in pieceToStringIdx.keys() \
                          if set(key) == set(translate[char] for char in val[0]))
      
  def getTopAndFront(self): return self.translate[0], self.translate[2]

def catchCubeError(s, log=True):
  try: return Cube(s).getResult()
  except AssertionError as e:
    if log: print(e)
    return [False,True]

def testCube():
  print(catchCubeError('UUUUUULU??RRRRRRRRUF?FFFFFFDDDDDDDDDLLFLLLLLLBBBBBBBBB'))
  # print(catchCubeError('BBBBBBBBBUUUUUUUUULLLLLLLLLFFFFFFFFFDDDDDDDDDRRRRRRRRR'))

  # print(catchCubeError('UUUUUUUUURFRRRRRRRFRFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBB'))
  # print(catchCubeError('UUUUUUUUUBFFRRRRRRFRRFFFFFFDDDDDDDDDLLLLLLLLLRBBBBBBBB'))

status = [0]*20
out = display(IPython.display.Pretty('status'), display_id=True)

testCube()

status

(True, True)


#### GUI

In [ ]:
def plotSquare(x,y,z, color):
  return go.Surface(x=x, y=y, z=z, showscale=False,
      colorscale=[[0, color], [1, color]])
      
def cubeGUI(cubestring=False):
  lettersToColors = {'U':'white',
                    'L':'orange',
                    'F':'green',
                    'R':'red',
                    'B':'blue',
                    'D':'yellow',
                     '?': 'grey'}
  U =     '''UUU
             UUU
             UUU'''
  M = '''LLL FRR BFF RBB
         LLL FFF RRR BBB
         LLL FFF RRR BBB'''
  D =     '''DDD
             DDD
             DDD'''

  if cubestring:
    U = cubestring[:9]
    M = ' '.join(cubestring[i + shift:i + shift + 3] for shift in [0,3,6] for i in [36, 18, 9, 45])
    # print(M)
    D = cubestring[27:36]

  print(''.join(U.split()) +\
        ''.join(M.split()[2::4]) +\
        ''.join(M.split()[1::4]) +\
        ''.join(D.split()) +\
        ''.join(M.split()[::4]) +\
        ''.join(M.split()[3::4])
        )

  data = [plotSquare(
      x = [i%3, i%3 + 1],
      y = [2 - i//3,  2 - i//3 + 1],
      z = [[3,3],
           [3,3]],
      color = lettersToColors[c]
      ) for i, c in enumerate(''.join(U.split()))]\
      + [plotSquare(
      x = [i%3, i%3 + 1],
      y = [i//3,  i//3 + 1],
      z = [[0,0],
           [0,0]],
      color = lettersToColors[c]
      ) for i, c in enumerate(''.join(D.split()))]\
      + [plotSquare(
      x = [0,0],
      y = [2 - i % 3, 2 - i % 3 + 1],
      z = [[2 - i//3, 2 - i//3 + 1],
           [2 - i//3, 2 - i//3 + 1]],
      color = lettersToColors[c]
      ) for i, c in enumerate(''.join(M.split()[::4]))]\
      + [plotSquare(
      x = [i % 3, i % 3 + 1],
      y = [0,0],
      z = [[2 - i//3 + 1, 2 - i//3 + 1],
           [2 - i//3, 2 - i//3]],
      color = lettersToColors[c]
      ) for i, c in enumerate(''.join(M.split()[1::4]))]\
      + [plotSquare(
      x = [3, 3],
      y = [i % 3, i % 3 + 1],
      z = [[2 - i//3, 2 - i//3 + 1],
           [2 - i//3, 2 - i//3 + 1]],
      color = lettersToColors[c]
      ) for i, c in enumerate(''.join(M.split()[2::4]))]\
      + [plotSquare(
      x = [2 - i % 3, 2 - i % 3 + 1],
      y = [3, 3],
      z = [[2 - i//3, 2 - i//3],
           [2 - i//3 + 1, 2 - i//3 + 1]],
      color = lettersToColors[c]
      ) for i, c in enumerate(''.join(M.split()[3::4]))]\
      
  fig = go.Figure(data=data).update_layout(
          scene = {
              'camera_eye': {"x": -1.5, "y": -1.5, "z": 1.5},
              "aspectratio": {"x": 1, "y": 1, "z": 1}
          })
  fig.show()

# cubeGUI('UFBBUFFBUBLLLRLRBLRDRUFDBRUDDFFDLDFBFRDBLRFURURLUBUDDL')
cubeGUI('RFBULLBLBRURDUFRDDUDURFRLDFUBDLRFLBLFBLBDUDFFDUURBLBRF')
# cubeGUI('BBBLDBULF??????????????????LFBBULBBB??????????????????')
# cubeGUI('BBBLDBULF??????????????????LFBBULBBB??????????????????')
# cubeGUI()